In [ ]:
import os
import re
import PIL
import sys
import json
import time
import timm
import math
import copy
import torch
import pickle
import logging
import fnmatch
import argparse
import itertools
import torchvision
import numpy as np
%matplotlib inline
import pandas as pd
# from apex import amp
import seaborn as sns
import albumentations
import torch.nn as nn
from PIL import Image
from pathlib import Path
from copy import deepcopy
import scikitplot as skplt
from sklearn import metrics
import torch.optim as optim
from timm import create_model
from datetime import datetime
from timm.data.loader import *
from torchvision import models
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.utils.data as data
# from cutmix.cutmix import CutMix
from torch.autograd import Variable
from tqdm import tqdm, tqdm_notebook
from torch.optim import lr_scheduler
#from pytorch_metric_learning import loss
import torch.utils.model_zoo as model_zoo
from timm.models.layers.activations import *
from timm.utils import accuracy, AverageMeter
%config InlineBackend.figure_format = 'retina'
# from apex.parallel import convert_syncbn_model
from timm.utils import ApexScaler, NativeScaler
# from cutmix.utils import CutMixCrossEntropyLoss
from timm.models.registry import register_model
from timm.models.vision_transformer import VisionTransformer

# import VIT_imagenet.VIT as v
from collections import OrderedDict, defaultdict
from torch.utils.tensorboard import SummaryWriter
from warmup_scheduler import GradualWarmupScheduler
from timm.models.resnet import resnet26d, resnet50d
from torchvision import transforms, models, datasets
from timm.models.helpers import build_model_with_cfg
from torch.utils.data.sampler import SubsetRandomSampler
from randaugment import RandAugment, ImageNetPolicy, Cutout
# from apex.parallel import DistributedDataParallel as ApexDDP
from timm.models.layers import DropPath, to_2tuple, trunc_normal_
from timm.data import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
from timm.loss import LabelSmoothingCrossEntropy, SoftTargetCrossEntropy
from timm.models.efficientnet import efficientnet_b0, efficientnet_b1, efficientnet_b2, efficientnet_b3
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score, roc_curve, auc, roc_auc_score
#from timm.data import Dataset, DatasetTar, RealLabelsImagenet, create_loader, Mixup, FastCollateMixup, AugMixDataset

In [ ]:
torch.backends.cudnn.benchmark = True
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

In [ ]:
imagenet_dir = '/scratch/work/public/imagenet/train'

device = torch.device("cuda:0")

batch_size = 50
num_epochs = 500
# lr = 0.001
lr = 1e-4
beta = 1
step_size = 130
img_size = 224
test_size = 224


mean = [0.485] 
std = [0.229]
num_workers = 4
# Define your transforms for the training and testing sets
data_transforms = {
    'train': transforms.Compose([
        transforms.CenterCrop(img_size),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ]),
    'val': transforms.Compose([
        transforms.CenterCrop(test_size),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ]),
    'test': transforms.Compose([
        transforms.CenterCrop(test_size),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ])
}
print(device)

## ImageNet

In [ ]:
train_split_size = 1000
val_split_size = 200

In [ ]:
imagenet_train = datasets.ImageFolder(imagenet_dir, transform=data_transforms['train']
                                          )


#split into train and val samples 
train_split = torch.utils.data.random_split(imagenet_train, 
                                            [train_split_size, len(imagenet_train)-train_split_size])[0]

val_split = torch.utils.data.random_split(imagenet_train, 
                                            [len(imagenet_train)-val_split_size, val_split_size])[1]


# get train and val dataloaders
train = torch.utils.data.DataLoader(train_split, batch_size = batch_size, shuffle=True,
                                    num_workers=num_workers)

val = torch.utils.data.DataLoader(val_split, batch_size = batch_size, shuffle=True,
                                    num_workers=num_workers)

#### Check Train

In [ ]:
type(train)

In [ ]:
len(train)

In [ ]:
batch0 = next(iter(train))
batch0

In [ ]:
len(batch0)

In [ ]:
image0 = batch0[0][0]
label0 = batch0[1][0]
print('image:',image0)
print('label:',label0)
print('image size:',image0.size())

#### Check Val

In [ ]:
type(val)

In [ ]:
len(val)

In [ ]:
batch1 = next(iter(val))
batch1

In [ ]:
len(batch1)

In [ ]:
image1 = batch1[0][0]
label1 = batch1[1][0]
print('image:',image1)
print('label:',label1)
print('image size:',image1.size())

### CIFAR 10

from https://github.com/jeonggunlee/Vision-Transformer-Study/blob/main/Vision_Transformer_Cifar10.ipynb

In [ ]:
patch = 4
batch_size = 50

In [ ]:
train_split_size = 10000
val_split_size = 2000

In [ ]:
# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.CenterCrop(32),
#     transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

cifar_train = torchvision.datasets.CIFAR10(root='../data', train=True, download=True, transform=transform_train)
cifar_test = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform=transform_test)

#split into train and val samples 
# cifar_train_split = torch.utils.data.random_split(cifar_train, 
#                                             [train_split_size, len(cifar_train)-train_split_size])[0]

# cifar_val_split = torch.utils.data.random_split(cifar_train, 
#                                             [len(cifar_train)-val_split_size, val_split_size])[1]

# train and val dataloaders
trainloader = torch.utils.data.DataLoader(cifar_train, batch_size=batch_size, shuffle=True, num_workers=num_workers)
testloader = torch.utils.data.DataLoader(cifar_test, batch_size=100, shuffle=False, num_workers=num_workers)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

#### CIFAR Check

In [ ]:
batch2 = next(iter(trainloader))
batch2

In [ ]:
len(batch2)

In [ ]:
batch2[0]

In [ ]:
batch2[0].size()

In [ ]:
batch2[1]

In [ ]:
batch2[1].size()

### CIFAR Model

In [ ]:
cifar_model = VisionTransformer(
                                img_size = 32,
                                patch_size = patch,
                                num_classes = 10,
                                embed_dim = 512,
                                depth = 6,
                                num_heads = 8,
#                                 mlp_dim = 512,
#                                 dropout = 0.1,
#                                 emb_dropout = 0.1
                                )
cifar_model

In [ ]:
for param in cifar_model.parameters():
    param.requires_grad = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cifar_model.parameters(), lr=lr,)
cifar_model.to(device)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
count = count_parameters(cifar_model)
print("The number of parameters of the model is:", count)

# Define Model

In [ ]:
model = VisionTransformer()
model

In [ ]:
for param in model.parameters():
    param.requires_grad = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr,)
model.to(device)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
count = count_parameters(model)
print("The number of parameters of the model is:", count)

In [ ]:
def train_model(model, dataloader, criterion, optimizer, num_epochs=100):
    
    total_loss = 0.0
    total_correct = 0
    total_targets = 0
    best_acc = 0

    for e in range(num_epochs):

        epoch_loss = 0.0
        epoch_correct = 0
        epoch_targets = 0

        for i, data in enumerate(dataloader, 0):
            # extract inputs and labels
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero parameter gradients
            optimizer.zero_grad()

            # set model to training mode
            model.train()

            # forward pass
            outputs = model(inputs)

            # compute loss
            loss = criterion(outputs, labels)

            # backprop
            loss.backward()

            # optimizer step
            optimizer.step()

            # getting outputs in true/false 
            _, preds = outputs.max(1)
    #         _, preds = torch.max(outputs, 1)


            # ------------ EVALUATION -----------

            # compute number of correct predictions
            correct = torch.sum(preds == labels.data)

            # update correct counters
            epoch_correct += correct
            total_correct += correct

            # update loss counters
            epoch_loss += loss.item()
            total_loss += loss.item()

            # update target counters
            epoch_targets += labels.size(0)
            total_targets += labels.size(0)


            # calculate batch accuracy
            batch_acc = correct / labels.size(0) * 100


            # display every five batches
            if i % 5 == 0:
                print('Epoch: {0}/{1}, Batch: {2}/{3}, Loss: {4}, Acc: {5} %'.format(e+1,
                                                                              num_epochs,
                                                                              i,
                                                                              len(dataloader),
                                                                              loss.item(),
                                                                              batch_acc       
                                                                             ))

        # calculate epoch metrics
        epoch_loss = epoch_loss / len(dataloader)
        epoch_acc = epoch_correct / epoch_targets * 100
        
        if epoch_acc > best_acc:
            best_acc = epoch_acc

        # calculate running accuracy
        running_acc = total_correct / total_targets * 100

        print('-'*20)
        print('Epoch {0} finished. Accuracy: {1} %     Loss: {2}'.format(e+1,
                                                                        epoch_acc,
                                                                        epoch_loss,
                                                                       ))
        print('Running Accuracy: {0} %'.format(running_acc))

        print('-'*20)


    total_acc = total_correct / total_targets *100

    print('Total accuracy: {0} %'.format(total_acc))
    print('Best accuracy: {0} %'.format(best_acc))
    
    return

In [ ]:
train_model(cifar_model, dataloader=trainloader, criterion=criterion, optimizer=optimizer, num_epochs=50)

In [ ]:
def test_model(model, dataloader, criterion):
    total_loss = 0.0
    total_correct = 0
    total_targets = 0
    best_acc = 0
    
    for i, data in enumerate(dataloader, 0):
        # extract inputs and labels
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # set model to eval mode
        model.eval()

        # forward pass
        outputs = model(inputs)

        # compute loss
        loss = criterion(outputs, labels)

        # getting outputs in true/false 
        _, preds = outputs.max(1)
    #         _, preds = torch.max(outputs, 1)


        # ------------ EVALUATION -----------

        # compute number of correct predictions
        correct = torch.sum(preds == labels.data)

        # update correct counter
        total_correct += correct

        # update loss counter
        total_loss += loss.item()

        # update target counter
        total_targets += labels.size(0)


        # calculate batch accuracy
        batch_acc = correct / labels.size(0) * 100


        # display every five batches
        if i % 5 == 0:
            print('Epoch: {0}/{1}, Batch: {2}/{3}, Loss: {4}, Acc: {5} %'.format('',
                                                                          '',
                                                                          i,
                                                                          len(dataloader),
                                                                          loss.item(),
                                                                          batch_acc       
                                                                         ))
        
        
    total_acc = total_correct / total_targets *100

    print('Total accuracy: {0} %'.format(total_acc))
    
    return

In [ ]:
test_model(cifar_model, dataloader=testloader, criterion=criterion)